In [4]:
import os
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe의 Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
input_video_path = "C:/Users/elena/Downloads/KETI_SL_0000001137.avi"
cap = cv2.VideoCapture(input_video_path)

# 새로운 동영상 파일 생성 준비
output_video_path = 'output_video.mp4'
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

idx = '열'

PATH = "C:/Users/elena/Desktop/data/"

data = []
while cap.isOpened():
    ret, frame = cap.read()  # 프레임 읽기
    if not ret: # 제대로 읽지 못했을 경우 ret=False
        break`
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 손 검출 수행
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(hand_landmarks.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
            
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n', # Einstein Summation 표기법으로 다차원 배열의 내적 계산 -> arccos값 구해서 angle에 할당
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], # einsum: 두 개의 벡터 배열 간의 내적 계산
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
            
            angle = np.degrees(angle) # Convert radian to degree
    
            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx)

            d = np.concatenate([joint.flatten(), angle_label])
            
            data.append(d)

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    # 결과를 새로운 동영상에 기록
    out.write(frame)

data = np.array(data)
print(idx, data.shape)
np.save('data.npy', data)

# seq_length = 30

# # 30이라는 사이즈의 윈도우를 가진 100개의 포인트
# full_seq_data = []
# for seq in range(len(data) - seq_length): # data의 길이에서 seq_length만큼 뺌
#     full_seq_data.append(data[seq:seq + seq_length]) # full_seq_data에 data의 제일 뒤 쪽 seq_length만큼의 frame을 append

# full_seq_data = np.array(full_seq_data)
# print(idx, full_seq_data.shape)
# np.save('C:/Users/Elena/Desktop/seq_data.npy', full_seq_data)

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()

열 (80, 100)
열 (50, 30, 100)


In [5]:
import numpy as np

data = np.load('data.npy')
print(data) # 총 97프레임 중 80프레임 인식

[['0.3686419427394867' '0.888746440410614' '1.3251178643258754e-07' ...
  '10.508908' '1.8008835' '열']
 ['0.3841209411621094' '0.8017615079879761' '2.633049689393374e-07' ...
  '13.775346' '5.593472' '열']
 ['0.5735070705413818' '0.8655163049697876' '5.24487617781233e-08' ...
  '5.763534' '1.2423095' '열']
 ...
 ['0.37256401777267456' '0.7951946258544922' '2.0155064817117818e-07' ...
  '9.403677' '1.8946248' '열']
 ['0.5844031572341919' '0.9408870339393616' '1.3918835861659318e-07' ...
  '4.599236' '5.6893225' '열']
 ['0.34350547194480896' '0.9066401124000549' '8.638599524601887e-08' ...
  '9.382911' '1.1706026' '열']]


## 왼손 오른손 구분해서 저장

In [34]:
import os
import cv2
import mediapipe as mp
import numpy as np


# Mediapipe의 Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# 동영상 파일 열기
input_video_path = "C:/Users/elena/Downloads/KETI_SL_0000001137.avi"
cap = cv2.VideoCapture(input_video_path)

# 새로운 동영상 파일 생성 준비
output_video_path = 'output_video.mp4'
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

idx = '열'

left_data = []
right_data = []
while cap.isOpened():
    ret, frame = cap.read()  # 프레임 읽기
    if not ret: # 제대로 읽지 못했을 경우 ret=False
        break
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 손 검출 수행
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(hand_landmarks.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
            
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n', # Einstein Summation 표기법으로 다차원 배열의 내적 계산 -> arccos값 구해서 angle에 할당
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], # einsum: 두 개의 벡터 배열 간의 내적 계산
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
            
            angle = np.degrees(angle) # Convert radian to degree
    
            angle_label = np.array([angle], dtype=np.float32)
            angle_label = np.append(angle_label, idx)

            d = np.concatenate([joint.flatten(), angle_label])
            
            is_left_hand = results.multi_handedness[0].classification[0].label == "Left"

            if is_left_hand:
                left_data.append(d)
            else:
                right_data.append(d)
            

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    # 결과를 새로운 동영상에 기록
    out.write(frame)

left_data = np.array(left_data)
right_data = np.array(right_data)
np.save('left_data.npy', left_data)
np.save('right_data.npy', right_data)
# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()

In [40]:
left = np.load('left_data.npy')
right = np.load('right_data.npy')

print(left.shape)
print(right.shape)

(64, 100)
(16, 100)
